In [ ]:
import vk
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from datetime import datetime
data = pd.DataFrame(columns=['ids', 'first_name', 'last_name', 'is_closed', 'bdate', 'sex', 'city_id', 'country_id' ])

In [ ]:
def token_init(token): 
    global vk_api
    session = vk.Session(access_token=token)
    vk_api = vk.API(session)

In [ ]:
def get_members(groupid): #create list of ids of reqired group
    first = vk_api.groups.getMembers(group_id=groupid, v=5.92)
    group_m = first["items"]  
    count = np.floor_divide(first["count"],1000)
    
    for i in range(1, count+1):  
        group_m = group_m + vk_api.groups.getMembers(group_id=groupid, v=5.92, offset=i*1000)["items"]
    return group_m

In [ ]:
def arrays_init(number):
    global ids_new
    ids_new = pd.Series(number)
    global first_names
    first_names = pd.Series(number, dtype=object)
    global last_names
    last_names=pd.Series(number, dtype=object)
    global bdates
    bdates = pd.Series(number, dtype=object)
    global sex 
    sex = pd.Series(number, dtype=np.int64)
    global city_ids
    city_ids = pd.Series(number, dtype=np.int64)
    global country_ids
    country_ids = pd.Series(number, dtype=np.int64)
    global is_closeds
    is_closeds = pd.Series(number,dtype=bool)#create list of indexes for providing users.get

In [ ]:
def index(ids_file):
    global data1
    data1 = []
    count= np.floor_divide(len(ids_file), 900)
    index=np.zeros((count+1,2))
    
    for i in range(count):
        index[(i,0)]=i*900
        index[(i,1)]=(i+1)*900
    index[(count,0)]=count*900
    index[(count,1)]=len(ids_file)
    
    for i in range(count+1):
        data1[int(index[(i,0)]):int(index[(i,1)])] = vk_api.users.get(user_ids=ids_file[int(index[(i,0)]):int(index[(i,1)])],fields = 'bdate, city, country, sex', v=5.92)

In [ ]:
def sorting(number):
    for i in range(number):
        try:
            ids_new[i] = data1[i]['id']
            first_names[i] = data1[i]['first_name']
            last_names[i] = data1[i]['last_name']
            sex[i] = data1[i]['sex']
            is_closeds[i] = data1[i]['is_closed']
            city_ids[i] = data1[i]['city']['id']
            country_ids[i] = data1[i]['country']['id']
            bdates[i]= data1[i]['bdate']
        except KeyError:
            pass

In [ ]:
def data_init():    
    data['ids']=ids_new
    data['first_name'] = first_names
    data['last_name'] = last_names
    data['sex'] = sex
    data['city_id'] = city_ids.astype(np.int16)
    data['country_id'] = country_ids.astype(np.int16)
    data['is_closed'] = is_closeds
    data['bdate'] = bdates

In [ ]:
def export_ids():
    f = open("ids.txt", "w")
    for i in range(len(ids)):
        f.write(str(ids[i]) + '\n')
    f.close()      

In [ ]:
def export_json():
    with open("jsonbase.json", "wb") as f:
                f.write(json.dumps(data1).encode("utf-8"))

In [ ]:
def export_data(file_format):
    if file_format == "csv":
            data.to_csv("database.csv", index = False)
    elif file_format == "excel":
            data.to_excel("database.xlsx", index = False)
    else:
            print("Wrong format")

In [ ]:
def deactivated_find(number):  
    with open('jsonbase.json', 'r+') as f:
        data1 = json.loads(f.read())
    
    global deactivated
    deactivated = np.zeros(number, dtype = np.int64)
    for i in range(number):
        try:
            if data1[i]['deactivated'] == "banned" or data1[i]['deactivated'] == "deleted":
                deactivated[i] = data1[i]['id']
        except KeyError:
            pass

In [ ]:
def  deactivated_sort(number):   
    for i in range(number):
        if deactivated[i] != 0:
            data.loc[i,'ids'] = 0

In [ ]:
def parse_bdate():
    date = datetime.now().year
    global bdata
    bdata = np.zeros(len(bdates), dtype = np.int32)
    for i in range(len(bdata)):
        if bdates.get_values()[i].count('.') > 1: # Filter by year existance
            user_byear = int(bdates.get_values()[i].split('.')[2]) # Get year from date string
            if user_byear > 1975: # Check year on correctness
                bdata[i] = user_byear
        else:
            pass
        if bdata[i] == 0:
            pass
        else:
            bdata[i] = date - bdata[i]

In [ ]:
def sort_bdate():
    global sorted_bdata
    np.ndarray.sort(bdata)
    parsed_bdata = bdata[np.array(np.nonzero(bdata))[0,0]:]
    temp_bdata = np.unique(parsed_bdata)
    sorted_bdata = np.zeros((2,len(np.unique(parsed_bdata))), dtype = np.int16)
    for i in range(len(temp_bdata)):
        sorted_bdata[0, i] = temp_bdata[i]
        sorted_bdata[1, i] = np.count_nonzero(parsed_bdata == temp_bdata[i])
    return sorted_bdata

In [ ]:
def user_info_fr_id(user_id):
    user_info = data.loc[data['ids'] == user_id]
    return user_info

In [ ]:
def demography_viz():
    fig = plt.figure()
    fig = plt.gcf()
    fig.set_size_inches(19.2, 10.8,forward = True)
    plt.grid()
    
    xtiks = np.arange(len(np.nonzero(bdata)[0]))
    ytiks = np.arange(0,130,1)
    plt.xticks(xtiks)
    plt.yticks(ytiks)
    plt.title('Demogracy', fontsize = 20)
    plt.xlabel('bdate age', fontsize = 20)
    plt.ylabel('bdate count',fontsize = 20)
    plt.plot(sorted_bdata[0], sorted_bdata[1], color='red',label= 'male')
    
    plt.legend();

In [ ]:
def control(token,group,number,format_file):
    global ids
    token_init(token)
    ids=get_members(group) #short name or id of required group
    arrays_init(number) #give file with ids
    index(ids) #give file with ids
    sorting(number) #give number of people to sort
    data_init() #give file with ids
    export_ids()
    export_json() 
    export_data(format_file)
    deactivated_find(number)
    deactivated_sort(number)
    parse_bdate()
    sort_bdate()
    

In [ ]:
if __name__ == "__main__":
    token = "" #enter your token
    group = ""#enter shorname or groupid(without "") of required group
    number = 123 #enter number of people needed to sort
    format_file = "csv" # csv or excel
    control(token, group, number, format_file)#enter groupid, number of people to sort, file format to save data 
    def demography_viz() #visualisation of demography rangement